In [ ]:
!pip install -q transformers
!pip install -q datasets

     |████████████████████████████████| 4.2 MB 6.9 MB/s 
     |████████████████████████████████| 596 kB 58.1 MB/s 
     |████████████████████████████████| 86 kB 4.1 MB/s 
     |████████████████████████████████| 6.6 MB 36.2 MB/s 
     |████████████████████████████████| 346 kB 6.6 MB/s 
     |████████████████████████████████| 140 kB 44.1 MB/s 
     |████████████████████████████████| 86 kB 6.4 MB/s 
     |████████████████████████████████| 212 kB 48.4 MB/s 
     |████████████████████████████████| 1.1 MB 51.0 MB/s 
     |████████████████████████████████| 127 kB 49.1 MB/s 
     |████████████████████████████████| 144 kB 44.7 MB/s 
     |████████████████████████████████| 271 kB 56.2 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 112 kB 48.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import random
import pickle

import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW

from tqdm.auto import tqdm

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TFAutoModelForSequenceClassification
from transformers import get_scheduler, TrainingArguments, Trainer, DefaultDataCollator

from datasets import Dataset, load_metric, list_metrics, load_from_disk

In [ ]:
path_to_drive = "drive/MyDrive/"

#CUDA

In [ ]:
# if CUDA
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
device

device(type='cpu')

# TPU

In [ ]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

# Loading test data

In [ ]:
test_tokenized = load_from_disk(path_to_drive + "nlp-tokenized/aita_balanced/test/")
test_tokenized.set_format("torch")

In [ ]:
eval_dataloader = DataLoader(test_tokenized, batch_size=8)

# Evaluating

In [ ]:
model = torch.load(path_to_drive + "nlp/models/aita_balanced/1/epoch2")

In [ ]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
accuracy = load_metric("accuracy")
f1 = load_metric("f1", average="weigthed")
f1_weighted = load_metric("f1", average=None)

auc = load_metric("roc_auc")

In [ ]:
preds_all = []
refs_all = []

In [ ]:
progress_bar = tqdm(range(len(eval_dataloader)))
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    preds_all.append(predictions)
    refs_all.append(batch["labels"])
    f1.add_batch(predictions=predictions, references=batch["labels"])
    f1b.add_batch(predictions=predictions, references=batch["labels"])
    accuracy.add_batch(predictions=predictions, references=batch["labels"])
    #auc.add_batch(predictions=predictions, references=batch["labels"])
    progress_bar.update(1)

  0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
accuracy.compute()

{'accuracy': 0.7531063618290258}

In [ ]:
f1results = f1b.compute(average=None)

ValueError: ignored

In [ ]:
f1results = np.array([0.58743633, 0.6838271 , 0.999001  , 0.68071242, 0.5243129 ,
        0.55550074, 0.98057028, 0.99576588])

In [ ]:
label_dict = {'asshole': 0,
 'everyone sucks': 1,
 'meta': 2,
 'no a-holes here': 3,
 'not enough info': 4,
 'not the a-hole': 5,
 'tl;dr': 6,
 'update': 7}

In [ ]:
pd.DataFrame({'x':label_dict.keys(), 'y':f1results})

,x,y
0,asshole,0.587436
1,everyone sucks,0.683827
2,meta,0.999001
3,no a-holes here,0.680712
4,not enough info,0.524313
5,not the a-hole,0.555501
6,tl;dr,0.980570
7,update,0.995766


In [ ]:
f1.compute(average="weighted")

{'f1': 0.7499159529910822}

# in one script

In [ ]:
evaluation_set = "teen" 
model_set = "teen"
model_number = "1"
num_epochs = 4

In [ ]:
test_tokenized = load_from_disk(path_to_drive + "nlp/data/restricted/tokenized/" + evaluation_set + "/test/")
test_tokenized.set_format("torch")
eval_dataloader = DataLoader(test_tokenized, batch_size=32)

progress_bar = tqdm(range(len(eval_dataloader) * num_epochs))
for epoch in range(1, num_epochs):
  model = torch.load(path_to_drive + "nlp/models/"+model_set+"/"+model_number+"/epoch" + str(epoch), map_location=torch.device('cpu'))
  model.eval()
  accuracy = load_metric("accuracy")
  precision = load_metric("precision", average="weigthed")
  recall = load_metric("recall", average="weigthed")
  f1_weighted = load_metric("f1", average="weigthed")
  f1 = load_metric("f1", average=None)
  probs_all = []

  for batch in eval_dataloader:
      batch = {k: v.to(device) for k, v in batch.items()}
      with torch.no_grad():
          outputs = model(**batch)
      logits = outputs.logits
      predictions = torch.argmax(logits, dim=-1)
      probs_all.append(logits)
      f1.add_batch(predictions=predictions, references=batch["labels"])
      f1_weighted.add_batch(predictions=predictions, references=batch["labels"])
      accuracy.add_batch(predictions=predictions, references=batch["labels"])
      precision.add_batch(predictions=predictions, references=batch["labels"])
      recall.add_batch(predictions=predictions, references=batch["labels"])
      progress_bar.update(1)
  
  eval_results = {"epoch": epoch,
                  "f1": f1.compute(average=None),
                  "f1_weighted": f1_weighted.compute(average="weighted"),
                  "accuracy": accuracy.compute(),
                  "precision": precision.compute(average="weighted"),
                  "recall": recall.compute(average="weighted"),
                  "probs": probs_all}
  eval_path = path_to_drive + "nlp/results/"+model_set+"/"+model_number+"/"+evaluation_set+"-epoch" + str(epoch)
  np.save(eval_path, eval_results)
  print("\nepoch:" + str(eval_results["epoch"]))
  print("f1:" + str(eval_results["f1"]))
  print("f1_weighted:" + str(eval_results["f1_weighted"]))
  print("precision:" + str(eval_results["precision"]))
  print("recall:" + str(eval_results["recall"]))
  print("accuracy:" + str(eval_results["accuracy"]))


  0%|          | 0/252 [00:00<?, ?it/s]

In [ ]:

read_dictionary = np.load('my_file.npy',allow_pickle='TRUE').item()

'[1 2]'